In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('frg20-21data.txt')
winter2 = pd.read_csv ('frg21-22data.txt')
winter3 = pd.read_csv ('frg22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
229,FRG,2021-12-09 02:53:00,T,-SN BR
661,FRG,2021-12-24 08:28:00,T,-SN
662,FRG,2021-12-24 08:32:00,T,-SN
663,FRG,2021-12-24 08:53:00,T,-SN
664,FRG,2021-12-24 08:58:00,T,-SN
...,...,...,...,...
2298,FRG,2022-02-14 00:53:00,T,-SN
2300,FRG,2022-02-14 01:53:00,T,-SN
2468,FRG,2022-02-19 18:53:00,T,-SN
2470,FRG,2022-02-19 20:53:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\3060002225.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated an

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
frg = LocationInfo(
    'Farmingdale', 'New York', 
    latitude= 40.7326,
    longitude= -73.4454, 
    timezone='America/New_York'
)

frg

LocationInfo(name='Farmingdale', region='New York', timezone='America/New_York', latitude=40.7326, longitude=-73.4454)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(frg.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(frg.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(frg.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:59:17  2020-12-01 16:26:36  2020-12-01
1   2020-12-02 07:00:17  2020-12-02 16:26:23  2020-12-02
2   2020-12-03 07:01:16  2020-12-03 16:26:12  2020-12-03
3   2020-12-04 07:02:14  2020-12-04 16:26:04  2020-12-04
4   2020-12-05 07:03:11  2020-12-05 16:25:58  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:35:02  2021-02-24 17:39:22  2021-02-24
86  2021-02-25 06:33:34  2021-02-25 17:40:32  2021-02-25
87  2021-02-26 06:32:05  2021-02-26 17:41:41  2021-02-26
88  2021-02-27 06:30:34  2021-02-27 17:42:50  2021-02-27
89  2021-02-28 06:29:03  2021-02-28 17:43:58  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:59:02  2021-12-01 16:26:39  2021-12-01
1   2021-12-02 07:00:02  2021-12-02 16:26:25  2021-12-02
2   2021-12-03 07:01:02  2021-12-03 16:26:14  2021-12-03
3   2021-12-04 07:02:00  2021-12-04 16:26:05  2021-12-04
4   2021-12-05 07:02:57  2021-12-05 16:25:59  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:35:24  2022-02-24 17:39:05  2022-02-24
86  2022-02-25 06:33:55  2022-02-25 17:40:15  2022-02-25
87  2022-02-26 06:32:26  2022-02-26 17:41:24  2022-02-26
88  2022-02-27 06:30:56  2022-02-27 17:42:33  2022-02-27
89  2022-02-28 06:29:25  2022-02-28 17:43:42  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:58:47  2022-12-01 16:26:42  2022-12-01
1   2022-12-02 06:59:48  2022-12-02 16:26:28  2022-12-02
2   2022-12-03 07:00:48  2022-12-03 16:26:16  2022-12-03
3   2022-12-04 07:01:46  2022-12-04 16:26:07  2022-12-04
4   2022-12-05 07:02:44  2022-12-05 16:26:00  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:35:45  2023-02-24 17:38:48  2023-02-24
86  2023-02-25 06:34:17  2023-02-25 17:39:58  2023-02-25
87  2023-02-26 06:32:48  2023-02-26 17:41:07  2023-02-26
88  2023-02-27 06:31:18  2023-02-27 17:42:16  2023-02-27
89  2023-02-28 06:29:47  2023-02-28 17:43:25  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FRG,2020-12-09 18:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:06:45,2020-12-09 16:25:56
1,FRG,2020-12-16 20:53:00,0.0001,-SN,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
2,FRG,2020-12-16 21:53:00,0.2500,-SN BR,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
3,FRG,2020-12-16 22:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
4,FRG,2020-12-16 23:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
...,...,...,...,...,...,...,...
80,FRG,2021-02-19 19:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:42:11,2021-02-19 17:33:30
81,FRG,2021-02-19 21:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:42:11,2021-02-19 17:33:30
82,FRG,2021-02-19 22:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:42:11,2021-02-19 17:33:30
83,FRG,2021-02-19 23:53:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:42:11,2021-02-19 17:33:30


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FRG,2021-12-09 02:53:00,0.0001,-SN BR,2021-12-09,2021-12-09 07:06:33,2021-12-09 16:25:55
1,FRG,2021-12-24 08:53:00,0.0001,-SN,2021-12-24,2021-12-24 07:16:08,2021-12-24 16:31:01
2,FRG,2022-01-17 22:53:00,0.0001,-SN,2022-01-17,2022-01-17 07:15:04,2022-01-17 16:53:07
3,FRG,2022-01-17 23:53:00,0.0001,-SN,2022-01-17,2022-01-17 07:15:04,2022-01-17 16:53:07
4,FRG,2022-01-20 14:53:00,0.2500,-SN BR,2022-01-20,2022-01-20 07:13:27,2022-01-20 16:56:37
5,FRG,2022-01-20 15:53:00,0.0001,-SN BR,2022-01-20,2022-01-20 07:13:27,2022-01-20 16:56:37
6,FRG,2022-01-20 16:53:00,0.0001,-SN BR,2022-01-20,2022-01-20 07:13:27,2022-01-20 16:56:37
7,FRG,2022-01-24 01:53:00,0.0001,-SN,2022-01-24,2022-01-24 07:10:49,2022-01-24 17:01:24
8,FRG,2022-01-24 09:53:00,0.0001,-SN BR,2022-01-24,2022-01-24 07:10:49,2022-01-24 17:01:24
9,FRG,2022-01-24 13:53:00,0.0001,-SN,2022-01-24,2022-01-24 07:10:49,2022-01-24 17:01:24


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FRG,2022-12-12 03:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:08:48,2022-12-12 16:26:14
1,FRG,2022-12-12 05:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:08:48,2022-12-12 16:26:14
2,FRG,2022-12-12 06:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:08:48,2022-12-12 16:26:14
3,FRG,2022-12-12 07:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:08:48,2022-12-12 16:26:14
4,FRG,2022-12-12 08:53:00,0.0001,-SN BR,2022-12-12,2022-12-12 07:08:48,2022-12-12 16:26:14
5,FRG,2023-01-23 19:53:00,0.0001,-SN,2023-01-23,2023-01-23 07:11:41,2023-01-23 16:59:54
6,FRG,2023-01-23 20:53:00,0.0001,-SN,2023-01-23,2023-01-23 07:11:41,2023-01-23 16:59:54
7,FRG,2023-01-23 21:53:00,0.0001,-SN,2023-01-23,2023-01-23 07:11:41,2023-01-23 16:59:54
8,FRG,2023-01-31 12:53:00,0.0001,-SN BR,2023-01-31,2023-01-31 07:05:11,2023-01-31 17:09:42
9,FRG,2023-02-01 09:53:00,0.0001,-SN,2023-02-01,2023-02-01 07:04:14,2023-02-01 17:10:56


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
12,FRG,2020-12-17 09:53:00,2.7900,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43
13,FRG,2020-12-17 15:53:00,0.0001,-SN,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43
15,FRG,2020-12-20 15:53:00,0.0001,-SN,2020-12-20,2020-12-20 07:14:23,2020-12-20 16:28:59
18,FRG,2021-01-20 12:53:00,0.0001,-SN,2021-01-20,2021-01-20 07:13:18,2021-01-20 16:56:54
19,FRG,2021-01-20 13:53:00,0.7600,-SN BR,2021-01-20,2021-01-20 07:13:18,2021-01-20 16:56:54
22,FRG,2021-01-27 09:53:00,0.0001,-SN,2021-01-27,2021-01-27 07:08:17,2021-01-27 17:05:22
30,FRG,2021-02-01 07:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32
31,FRG,2021-02-01 08:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32
32,FRG,2021-02-01 09:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32
33,FRG,2021-02-01 10:53:00,0.7600,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
12,FRG,2020-12-17 09:53:00,2.7900,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43,2020-12-17 08:12:40,2020-12-17 15:27:43
18,FRG,2021-01-20 12:53:00,0.0001,-SN,2021-01-20,2021-01-20 07:13:18,2021-01-20 16:56:54,2021-01-20 08:13:18,2021-01-20 15:56:54
19,FRG,2021-01-20 13:53:00,0.7600,-SN BR,2021-01-20,2021-01-20 07:13:18,2021-01-20 16:56:54,2021-01-20 08:13:18,2021-01-20 15:56:54
22,FRG,2021-01-27 09:53:00,0.0001,-SN,2021-01-27,2021-01-27 07:08:17,2021-01-27 17:05:22,2021-01-27 08:08:17,2021-01-27 16:05:22
31,FRG,2021-02-01 08:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32,2021-02-01 08:03:46,2021-02-01 16:11:32
32,FRG,2021-02-01 09:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32,2021-02-01 08:03:46,2021-02-01 16:11:32
33,FRG,2021-02-01 10:53:00,0.7600,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32,2021-02-01 08:03:46,2021-02-01 16:11:32
34,FRG,2021-02-01 11:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32,2021-02-01 08:03:46,2021-02-01 16:11:32
35,FRG,2021-02-01 13:53:00,0.0001,-SN BR,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32,2021-02-01 08:03:46,2021-02-01 16:11:32
36,FRG,2021-02-01 14:53:00,0.7600,SN FZFG,2021-02-01,2021-02-01 07:03:46,2021-02-01 17:11:32,2021-02-01 08:03:46,2021-02-01 16:11:32


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.395252
2021-2022: 0.07776923076923076
2022-2023: 0.0001


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,FRG,2020-12-09 18:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:06:45,2020-12-09 16:25:56
1,FRG,2020-12-16 20:53:00,0.0001,-SN,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
2,FRG,2020-12-16 21:53:00,0.2500,-SN BR,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
3,FRG,2020-12-16 22:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
4,FRG,2020-12-16 23:53:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:12:02,2020-12-16 16:27:22
5,FRG,2020-12-17 00:53:00,0.5100,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43
6,FRG,2020-12-17 01:53:00,0.5100,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43
7,FRG,2020-12-17 02:53:00,0.0001,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43
8,FRG,2020-12-17 03:53:00,0.2500,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43
9,FRG,2020-12-17 04:53:00,2.2900,-SN BR,2020-12-17,2020-12-17 07:12:40,2020-12-17 16:27:43


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_43404\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.5630782608695651
2021-2022: 0.0001
2022-2023: 0.0001


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.frg.csv', index=False)